# Multinet implementation for writer identification

In [1]:
import pgmpy
import pandas as pd
import numpy as np
eps = 0.0000001

Reading the csv file

In [3]:
dataset = pd.read_csv("AND_Features.csv")
data = dataset.as_matrix()

Extracting writer id's from the ImageId column

In [4]:
#Constructing dataset for Positive samples
image_ids =  np.array(dataset['ImageId'])
writer_ids = []
for id in image_ids:
    writer_ids +=  [id[:4]]
print(writer_ids)

['0359', '0577', '1120', '1120', '1120', '1121', '1121', '1121', '1229', '1302', '1302', '1302', '1305', '1305', '1305', '1386', '1386', '1386', '1391', '1489', '1489', '1517', '1517', '0331', '0367', '0367', '0367', '0379', '0458', '0565', '0565', '0565', '1254', '1362', '1362', '1446', '1446', '1446', '1454', '1454', '1455', '1455', '1455', '1510', '1510', '1534', '1540', '1300', '1300', '1300', '1309', '1453', '1453', '1453', '1482', '1518', '1543', '0584', '0584', '0584', '1127', '1127', '1127', '1274', '1284', '1360', '1364', '1490', '1491', '1491', '0321', '0451', '0503', '0503', '0561', '0561', '0571', '0600', '0600', '0600', '1137', '1230', '1230', '1248', '1248', '1248', '1335', '1378', '1378', '1432', '1481', '1481', '1481', '1487', '1512', '1512', '1528', '1565', '0338', '0422', '0422', '0422', '0472', '1227', '1382', '1445', '1445', '1445', '1486', '1510', '1545', '1545', '1545', '0379', '1282', '1289', '1289', '1289', '1452', '1565', '1565', '0552', '0552', '0552', '1261',

Constructing all possible positive pairs and all possible negative pairs dataset

In [5]:
data = dataset.as_matrix()

positive_pairs_dataset = []
negative_pairs_dataset = []
for writer_index in range(len(writer_ids)):
    for same_writer_index in range(len(writer_ids)):
        if (writer_ids[writer_index] == writer_ids[same_writer_index]) and (writer_index != same_writer_index) :
            writer_features = data[writer_index,2:]
            positive_pairs_dataset += [[writer_ids[writer_index],data[writer_index,2:],data[same_writer_index,2:]]]

for writer_index in range(len(writer_ids)):
    for diff_writer_index in range(len(writer_ids)):
        if (writer_ids[writer_index] != writer_ids[diff_writer_index]):
            writer_features = data[writer_index,2:]
            negative_pairs_dataset += [[writer_ids[writer_index],writer_ids[diff_writer_index],data[writer_index,2:],data[diff_writer_index,2:]]]            
            
pos_df = pd.DataFrame(np.array(positive_pairs_dataset),columns = ['writer','features 1','features 2'])
neg_df = pd.DataFrame(np.array(negative_pairs_dataset),columns = ['writer1','writer2','features 1','features 2'])

print(pos_df)
print("--------------------------------------------------------------------------------------------")
print(neg_df.shape)

     writer                   features 1                   features 2
0      0359  [2, 1, 1, 0, 2, 2, 0, 2, 2]  [3, 2, 1, 0, 2, 2, 3, 0, 2]
1      0577  [2, 1, 1, 0, 2, 2, 0, 1, 2]  [2, 1, 0, 3, 2, 2, 1, 2, 2]
2      0577  [2, 1, 1, 0, 2, 2, 0, 1, 2]  [1, 1, 1, 1, 2, 3, 0, 0, 2]
3      1120  [2, 1, 1, 3, 2, 2, 0, 2, 2]  [1, 1, 1, 0, 2, 2, 0, 2, 2]
4      1120  [2, 1, 1, 3, 2, 2, 0, 2, 2]  [2, 1, 1, 0, 2, 2, 0, 0, 2]
5      1120  [1, 1, 1, 0, 2, 2, 0, 2, 2]  [2, 1, 1, 3, 2, 2, 0, 2, 2]
6      1120  [1, 1, 1, 0, 2, 2, 0, 2, 2]  [2, 1, 1, 0, 2, 2, 0, 0, 2]
7      1120  [2, 1, 1, 0, 2, 2, 0, 0, 2]  [2, 1, 1, 3, 2, 2, 0, 2, 2]
8      1120  [2, 1, 1, 0, 2, 2, 0, 0, 2]  [1, 1, 1, 0, 2, 2, 0, 2, 2]
9      1121  [2, 1, 1, 3, 2, 2, 0, 1, 2]  [2, 1, 1, 0, 2, 2, 0, 3, 2]
10     1121  [2, 1, 1, 3, 2, 2, 0, 1, 2]  [1, 1, 1, 0, 2, 2, 0, 1, 2]
11     1121  [2, 1, 1, 0, 2, 2, 0, 3, 2]  [2, 1, 1, 3, 2, 2, 0, 1, 2]
12     1121  [2, 1, 1, 0, 2, 2, 0, 3, 2]  [1, 1, 1, 0, 2, 2, 0, 1, 2]
13     1121  [1, 1, 

Building Dataframe with 18 columns for Bayesian Structure Learning.
Since there are 1 million all possible negative pairs, we choose randomly 5000 to go ahead as negative dataset 

In [6]:
Bayesian_pos_dataset = []
for i in range(len(positive_pairs_dataset)):
    row = []
    for j in range(9):
        row += [positive_pairs_dataset[i][1][j]]
    for k in range(9):
        row += [positive_pairs_dataset[i][2][k]]
    print(row)
    Bayesian_pos_dataset += [row]

Bayesian_neg_dataset = []
# choose randomly 5000 indices
for i in np.random.randint(neg_df.shape[0],size = 5000):
    row = []
    for j in range(9):
        row += [negative_pairs_dataset[i][2][j]]
    for k in range(9):
        row += [negative_pairs_dataset[i][3][k]]
    print(row)
    Bayesian_neg_dataset += [row]
print(np.array(Bayesian_dataset).shape)

[2, 1, 1, 0, 2, 2, 0, 2, 2, 3, 2, 1, 0, 2, 2, 3, 0, 2]
[2, 1, 1, 0, 2, 2, 0, 1, 2, 2, 1, 0, 3, 2, 2, 1, 2, 2]
[2, 1, 1, 0, 2, 2, 0, 1, 2, 1, 1, 1, 1, 2, 3, 0, 0, 2]
[2, 1, 1, 3, 2, 2, 0, 2, 2, 1, 1, 1, 0, 2, 2, 0, 2, 2]
[2, 1, 1, 3, 2, 2, 0, 2, 2, 2, 1, 1, 0, 2, 2, 0, 0, 2]
[1, 1, 1, 0, 2, 2, 0, 2, 2, 2, 1, 1, 3, 2, 2, 0, 2, 2]
[1, 1, 1, 0, 2, 2, 0, 2, 2, 2, 1, 1, 0, 2, 2, 0, 0, 2]
[2, 1, 1, 0, 2, 2, 0, 0, 2, 2, 1, 1, 3, 2, 2, 0, 2, 2]
[2, 1, 1, 0, 2, 2, 0, 0, 2, 1, 1, 1, 0, 2, 2, 0, 2, 2]
[2, 1, 1, 3, 2, 2, 0, 1, 2, 2, 1, 1, 0, 2, 2, 0, 3, 2]
[2, 1, 1, 3, 2, 2, 0, 1, 2, 1, 1, 1, 0, 2, 2, 0, 1, 2]
[2, 1, 1, 0, 2, 2, 0, 3, 2, 2, 1, 1, 3, 2, 2, 0, 1, 2]
[2, 1, 1, 0, 2, 2, 0, 3, 2, 1, 1, 1, 0, 2, 2, 0, 1, 2]
[1, 1, 1, 0, 2, 2, 0, 1, 2, 2, 1, 1, 3, 2, 2, 0, 1, 2]
[1, 1, 1, 0, 2, 2, 0, 1, 2, 2, 1, 1, 0, 2, 2, 0, 3, 2]
[1, 1, 1, 3, 2, 2, 0, 2, 2, 2, 1, 0, 0, 2, 2, 1, 2, 2]
[2, 1, 1, 3, 2, 2, 0, 0, 2, 3, 1, 1, 0, 2, 2, 0, 2, 2]
[2, 1, 1, 3, 2, 2, 0, 0, 2, 2, 1, 1, 3, 2, 2, 0, 3, 2]
[3, 1, 1, 

[0, 2, 1, 0, 1, 2, 1, 3, 2, 1, 1, 0, 4, 2, 2, 0, 3, 2]
[2, 0, 1, 3, 3, 2, 1, 4, 2, 1, 1, 1, 1, 2, 3, 0, 4, 2]
[2, 2, 0, 0, 2, 1, 3, 3, 2, 1, 4, 1, 3, 2, 2, 0, 1, 2]
[2, 2, 0, 0, 2, 1, 3, 3, 2, 2, 1, 1, 3, 2, 2, 3, 0, 2]
[3, 1, 2, 1, 0, 2, 1, 3, 2, 2, 1, 1, 1, 2, 1, 3, 0, 2]
[3, 1, 2, 1, 0, 2, 1, 3, 2, 1, 1, 1, 0, 1, 1, 0, 0, 2]
[0, 0, 1, 1, 2, 3, 2, 3, 2, 0, 1, 1, 1, 0, 2, 0, 2, 1]
[0, 0, 1, 1, 2, 3, 2, 3, 2, 0, 1, 0, 1, 0, 2, 0, 1, 1]
[0, 1, 0, 1, 0, 2, 0, 1, 1, 0, 1, 1, 1, 0, 2, 0, 2, 1]
[0, 1, 0, 1, 0, 2, 0, 1, 1, 0, 0, 1, 1, 2, 3, 2, 3, 2]
[3, 4, 1, 4, 2, 3, 1, 3, 2, 0, 2, 1, 0, 0, 2, 2, 2, 1]
[1, 1, 0, 1, 0, 1, 0, 2, 1, 0, 1, 1, 0, 2, 2, 0, 2, 2]
[1, 1, 0, 1, 0, 1, 0, 2, 1, 2, 2, 1, 3, 2, 2, 0, 0, 2]
[1, 1, 1, 2, 0, 1, 1, 4, 1, 3, 4, 2, 2, 0, 2, 1, 3, 2]
[1, 1, 1, 2, 0, 1, 1, 4, 1, 0, 1, 2, 4, 2, 2, 2, 2, 2]
[2, 3, 1, 1, 2, 2, 3, 4, 2, 3, 2, 1, 2, 0, 2, 1, 1, 1]
[2, 3, 1, 1, 2, 2, 3, 4, 2, 2, 0, 1, 1, 0, 2, 3, 0, 1]
[3, 2, 1, 2, 0, 2, 1, 1, 1, 2, 3, 1, 1, 2, 2, 3, 4, 2]
[3, 2, 1, 

[2, 0, 1, 1, 2, 2, 0, 4, 2, 2, 4, 2, 4, 3, 3, 3, 4, 2]
[3, 1, 1, 3, 2, 2, 0, 4, 2, 1, 0, 1, 1, 2, 2, 0, 3, 2]
[2, 0, 1, 0, 2, 2, 1, 2, 2, 3, 2, 1, 0, 2, 2, 0, 1, 2]
[3, 1, 1, 3, 3, 1, 3, 1, 2, 2, 1, 1, 3, 1, 2, 0, 1, 2]
[1, 0, 1, 1, 2, 2, 3, 0, 2, 3, 4, 1, 1, 2, 3, 1, 3, 2]
[1, 1, 1, 0, 0, 2, 2, 1, 1, 1, 1, 1, 3, 0, 1, 1, 2, 1]
[2, 1, 1, 3, 0, 2, 2, 0, 1, 3, 1, 0, 2, 2, 3, 0, 3, 2]
[3, 2, 1, 0, 2, 2, 1, 4, 2, 3, 0, 1, 0, 2, 1, 0, 4, 2]
[2, 1, 1, 0, 2, 0, 0, 3, 2, 1, 2, 1, 0, 0, 2, 0, 4, 2]
[3, 1, 1, 1, 2, 2, 1, 2, 1, 0, 1, 1, 1, 2, 3, 1, 2, 1]
[2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2, 2]
[3, 2, 1, 0, 2, 2, 0, 1, 2, 3, 0, 1, 0, 2, 2, 1, 2, 2]
[0, 1, 0, 2, 2, 2, 0, 1, 2, 1, 0, 1, 0, 2, 2, 0, 1, 2]
[1, 1, 0, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 0, 2, 2]
[1, 2, 1, 3, 2, 2, 1, 3, 2, 2, 2, 1, 0, 2, 2, 1, 1, 2]
[1, 1, 1, 3, 2, 2, 0, 0, 2, 2, 1, 1, 3, 2, 3, 0, 3, 2]
[3, 1, 1, 3, 2, 3, 0, 4, 2, 1, 1, 1, 1, 2, 3, 0, 2, 1]
[1, 2, 1, 0, 2, 2, 2, 0, 1, 3, 0, 1, 3, 2, 2, 0, 3, 2]
[2, 0, 1, 

[3, 1, 1, 1, 2, 1, 1, 3, 2, 3, 1, 1, 3, 2, 3, 3, 4, 2]
[0, 4, 2, 4, 2, 2, 0, 3, 2, 3, 1, 1, 0, 2, 2, 0, 2, 2]
[2, 2, 1, 1, 2, 2, 1, 2, 2, 3, 1, 1, 3, 2, 2, 0, 4, 2]
[3, 4, 1, 3, 2, 3, 0, 0, 2, 2, 2, 1, 3, 2, 2, 0, 4, 2]
[0, 1, 1, 0, 2, 1, 0, 2, 1, 1, 1, 1, 0, 0, 2, 1, 1, 1]
[1, 1, 0, 1, 0, 1, 1, 0, 1, 2, 1, 0, 2, 2, 2, 0, 1, 2]
[2, 1, 1, 2, 2, 2, 1, 4, 2, 1, 1, 1, 3, 0, 2, 0, 1, 2]
[3, 4, 1, 0, 2, 3, 0, 3, 2, 0, 1, 1, 3, 2, 2, 0, 0, 2]
[2, 1, 1, 1, 0, 1, 1, 2, 1, 0, 4, 1, 3, 2, 2, 1, 2, 2]
[0, 0, 1, 0, 2, 2, 0, 2, 2, 2, 1, 1, 0, 2, 3, 3, 2, 1]
[2, 1, 1, 0, 2, 2, 1, 0, 2, 3, 0, 1, 3, 0, 1, 0, 3, 2]
[3, 4, 1, 3, 2, 2, 1, 2, 2, 0, 1, 1, 0, 2, 1, 1, 0, 1]
[2, 1, 1, 3, 0, 2, 0, 3, 2, 1, 1, 1, 0, 2, 2, 0, 0, 2]
[2, 0, 1, 1, 2, 2, 3, 2, 2, 2, 1, 0, 3, 2, 1, 0, 3, 2]
[3, 4, 0, 1, 2, 2, 0, 1, 2, 1, 1, 1, 0, 2, 1, 0, 1, 2]
[2, 4, 2, 4, 3, 3, 3, 4, 2, 1, 1, 1, 1, 2, 3, 0, 0, 2]
[1, 0, 1, 3, 0, 1, 0, 3, 2, 3, 4, 1, 4, 2, 3, 1, 3, 2]
[1, 0, 1, 1, 2, 2, 0, 3, 2, 1, 1, 2, 4, 3, 2, 1, 2, 1]
[3, 1, 1, 

NameError: name 'Bayesian_dataset' is not defined

In [7]:
bay_pos_df = pd.DataFrame(Bayesian_pos_dataset,columns = ['f01','f02','f03','f04','f05','f06','f07','f08','f09','f11','f12','f13','f14','f15','f16','f17','f18','f19'])
bay_neg_df = pd.DataFrame(Bayesian_neg_dataset,columns = ['f01','f02','f03','f04','f05','f06','f07','f08','f09','f11','f12','f13','f14','f15','f16','f17','f18','f19'])

print(bay_neg_df)
categories_pos = {}
for col in bay_pos_df:
    categories_pos[col] = np.sort(bay_pos_df[col].unique())
    
categories_neg = {}
for col in bay_neg_df:
    categories_neg[col] = np.sort(bay_neg_df[col].unique())
    
print(categories_neg)

      f01  f02  f03  f04  f05  f06  f07  f08  f09  f11  f12  f13  f14  f15  \
0       2    1    1    2    2    2    0    2    2    1    1    0    1    2   
1       1    1    1    3    2    2    0    0    2    2    1    1    0    0   
2       3    0    1    1    2    2    1    1    2    1    1    1    0    2   
3       2    1    0    3    0    2    1    0    2    3    0    1    0    2   
4       1    2    1    3    2    1    1    1    1    2    2    1    0    0   
5       2    0    1    0    2    1    0    3    2    2    1    0    2    2   
6       2    1    0    4    0    2    0    3    2    1    0    1    3    2   
7       3    2    1    3    1    2    3    2    2    0    1    1    3    2   
8       0    1    2    4    2    2    0    3    2    1    1    1    1    0   
9       2    1    1    1    2    2    0    1    1    2    1    1    0    1   
10      2    1    1    0    2    1    0    0    2    1    1    1    1    0   
11      1    1    0    1    0    3    0    2    1    0    4    1

Using Hill Climbing to find appropriate 2 Bayesion Networks (One for same writer another for different writer)<br>
# Recommended : SKIP THIS STEP. DIRECTLY READ ALREADY WRITTEN MODELS
# [NOTE] : 
#    1) It takes a lot of time to run due to large size of input.<br>
#    2) 3 scores are used for each bayesian network. At the time of execution it is advisable to run only one hill climbing estimator not 6 at a time.Rest should be commented out.<br>
#    3) All 6 estimator were ran and best ones for both BN were chosen and written in a file. To skip this step directly read the models from file<br>
    

In [120]:
# BUILDING BAYESIAN NETWORK 

from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BicScore,BdeuScore,K2Score
#Comment this if not running this.
hc_BIC = HillClimbSearch(bay_pos_df, scoring_method=BicScore(bay_pos_df))
best_model_BIC_pos = hc_BIC.estimate()

#Comment this if not running this.
hc_BD = HillClimbSearch(bay_pos_df, scoring_method=BdeuScore(bay_pos_df))
best_model_Bdeu_pos = hc_BD.estimate()

#Comment this if not running this.
hc_K2 = HillClimbSearch(bay_pos_df, scoring_method=K2Score(bay_pos_df))
best_model_K2_pos = hc_K2.estimate()


#-----------------------------------------------------------
#Comment this if not running this.
hc_BIC = HillClimbSearch(bay_neg_df, scoring_method=BicScore(bay_neg_df))
best_model_BIC_neg = hc_BIC.estimate()

#Comment this if not running this.
hc_BD = HillClimbSearch(bay_neg_df, scoring_method=BdeuScore(bay_neg_df))
best_model_Bdeu_neg = hc_BD.estimate()

#Comment this if not running this.
hc_K2 = HillClimbSearch(bay_neg_df, scoring_method=BicScore(bay_neg_df))
best_model_K2_neg = hc_K2.estimate()
print(best_model_K2_neg.edges())

[('f03', 'f05'), ('f03', 'f08'), ('f03', 'f06'), ('f04', 'f09'), ('f04', 'f08'), ('f04', 'f07'), ('f04', 'f03'), ('f13', 'f18'), ('f13', 'f19'), ('f13', 'f17'), ('f13', 'f16'), ('f13', 'f15'), ('f19', 'f16'), ('f19', 'f11'), ('f19', 'f18'), ('f19', 'f12'), ('f19', 'f17'), ('f19', 'f15'), ('f08', 'f09'), ('f06', 'f02'), ('f09', 'f06'), ('f09', 'f05'), ('f09', 'f07'), ('f09', 'f01'), ('f09', 'f02'), ('f16', 'f12'), ('f14', 'f19'), ('f14', 'f13')]


Write the models in file. [SKIP THIS IF ABOVE CELL IS ALSO SKIPPED]

In [159]:
from pgmpy.readwrite import BIFReader, BIFWriter
writer_no = BIFWriter(best_model_K2_neg)
writer_no.write_bif(filename = 'model_no.bif')
writer_yes = BIFWriter(best_model_K2_pos)
writer_yes.write_bif(filename = 'model_yes.bif')

Read already written model from file [DIRECTLY EXECUTE THIS IF BAYESIAN ESTIMATION IS SKIPPED]

In [8]:
from pgmpy.readwrite import BIFReader, BIFWriter
reader_yes = BIFReader("model_yes.bif")
reader_no  = BIFReader("model_no.bif")

model_yes = reader_yes.get_model()
model_no  = reader_no.get_model()



# Below code is only for drawing the model , it doesn't affect then execution so should not be executed. It was used to generate model graphs used in the report


In [136]:
from graphviz import Digraph
dot = Digraph()
best_model =  best_model_K2_neg
for i in range(1,10) :
    dot.node('f0'+str(i),'f0'+str(i))
for i in range(1,10) :
    dot.node('f1'+str(i),'f1'+str(i))

for edge in best_model.edges():
    v1 = edge[0]
    v2 = edge[1]
    dot.edge(v1,v2)
dot.render('test-output/bayesian_no_K2_1.gv', view=True) 

f01 f01
f02 f02
f03 f03
f04 f04
f05 f05
f06 f06
f07 f07
f08 f08
f09 f09


'test-output/bayesian_no_K2_1.gv.pdf'

Fit the data to respective BN's and print their CPD's

In [9]:
from pgmpy.estimators import MaximumLikelihoodEstimator

model_yes.fit(bay_pos_df, estimator=MaximumLikelihoodEstimator)
model_no.fit(bay_neg_df, estimator=MaximumLikelihoodEstimator)

parents_yes = reader_yes.get_parents()
parents_no = reader_no.get_parents()

for cpd in model_yes.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    print(parents_yes[cpd.variable])
for cpd in model_no.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    print(parents_no[cpd.variable])
    print(cpd.values.shape)

CPD of f01:
╒════════╤═════════╤═════════════════════╤═════════════════════╤═════════════════════╤════════╤═════════════════════╕
│ f03    │ f03(0)  │ f03(0)              │ f03(1)              │ f03(1)              │ f03(2) │ f03(2)              │
├────────┼─────────┼─────────────────────┼─────────────────────┼─────────────────────┼────────┼─────────────────────┤
│ f09    │ f09(1)  │ f09(2)              │ f09(1)              │ f09(2)              │ f09(1) │ f09(2)              │
├────────┼─────────┼─────────────────────┼─────────────────────┼─────────────────────┼────────┼─────────────────────┤
│ f01(0) │ 0.3125  │ 0.0958904109589041  │ 0.19655172413793104 │ 0.09897292250233426 │ 0.0    │ 0.10526315789473684 │
├────────┼─────────┼─────────────────────┼─────────────────────┼─────────────────────┼────────┼─────────────────────┤
│ f01(1) │ 0.59375 │ 0.22602739726027396 │ 0.4241379310344828  │ 0.2642390289449113  │ 0.0    │ 0.15789473684210525 │
├────────┼─────────┼─────────────────────┼──

Function to classify two features<br>
Computes joint probabilty using the product of cpd's for observed values of features from two given writers<br>
Calculates joint probabilty from Bayesian Network for same writer<br>
Calculates joint probabilty from Bayesian Network for different writer<br>
If log of ratio of the two probablity is positive , then the features are from same writer else they are not<br>

In [51]:
def indexOf(writer1, writer2,categories):
    f = {}
    for i in range(1,len(writer1)+1):
        key = 'f0'+str(i)
        cat = categories[key].tolist()
        if writer1[i-1] not in cat:
            print(writer1[i-1] , " : " ,cat, " : ",key)
        else: f[key]= cat.index(writer1[i-1]) 

    for i in range(1,len(writer2)+1):
        key = 'f1'+str(i)
        cat = categories[key].tolist()
        if writer2[i-1] not in cat:
            print(writer2[i-1] , " : " ,cat, " : ",key)
        else: f[key]= cat.index(writer2[i-1])
    return f     

def classify(writer1,writer2):

    f1 = indexOf(writer1,writer2,categories_pos)
    f2 = indexOf(writer1,writer2,categories_neg)
    prod_yes = 1
    for cpd in model_yes.get_cpds():
        if len(parents_yes[cpd.variable]) == 2:
            prod_yes *= cpd.values[f1[cpd.variable]][f1[parents_yes[cpd.variable][0]]] [f1[parents_yes[cpd.variable][1]]]
        if len(parents_yes[cpd.variable]) == 1:
            prod_yes *= cpd.values[f1[cpd.variable]][f1[parents_yes[cpd.variable][0]]]
        if len(parents_yes[cpd.variable]) == 0:
            prod_yes *= cpd.values[f1[cpd.variable]]

    prod_no = 1
    for cpd in model_no.get_cpds():
        if len(parents_no[cpd.variable]) == 3:
            prod_no *= cpd.values[f2[cpd.variable]][f2[parents_no[cpd.variable][0]]][f2[parents_no[cpd.variable][1]]][f2[parents_no[cpd.variable][2]]]
        if len(parents_no[cpd.variable]) == 2:
            prod_no *= cpd.values[f2[cpd.variable]][f2[parents_no[cpd.variable][0]]] [f2[parents_no[cpd.variable][1]]]
        if len(parents_no[cpd.variable]) == 1:
            prod_no *= cpd.values[f2[cpd.variable]][f2[parents_no[cpd.variable][0]]]
        if len(parents_no[cpd.variable]) == 0:
            prod_no *= cpd.values[f2[cpd.variable]]

    
    ratio = prod_yes/prod_no
    logL  = np.log(ratio + eps)
    cls = 0
    if (logL > 0):
        cls =  1
    return [cls,logL,prod_yes,prod_no]
    

Testing for randomly shuffled for testing samples for multiple runs<br>
Average accuracy is the mean of the accuracies of all the runs

In [45]:
num_runs = 10
acc = []
for run_count in range(num_runs):
    # Randomly generate two testing dataset one for same writer , other for diff writer
    test_dataset_same = []
    test_dataset_diff = []

    for i in np.random.randint(len(positive_pairs_dataset),size = 500 ):
        test_dataset_same += [[positive_pairs_dataset[i][1],positive_pairs_dataset[i][2]]]

    for i in np.random.randint(len(negative_pairs_dataset)-1,size = 1000):
        test_dataset_diff += [[negative_pairs_dataset[i][2],negative_pairs_dataset[i][3]]]

    correct_classification_same = 0
    for pos_samples in test_dataset_same:
        result = classify(pos_samples[0],pos_samples[1])
        print(result[1])
        if result[0]==1:
            correct_classification_same += 1
    same_acc = correct_classification_same/500
    print("Accuracy when writers are same :", same_acc)
    
    correct_classification_diff  = 0
    for neg_samples in test_dataset_diff:
        result = classify(neg_samples[0],neg_samples[1])
        if result[0]==0:
            correct_classification_diff += 1
    diff_acc = correct_classification_diff/1000
    print("Accuracy when writers are diff :", diff_acc)
    accuracy = (correct_classification_diff +correct_classification_same)/1500
    acc += [round(accuracy*1000)/1000]
    print("Overall Accuracy for run ",run_count,"   :",accuracy,"\n\n")
print("Accuracy over runs : ",acc)
avg = np.average(acc)
print("Average accuracy: ",avg*100,"%")

-0.148833259247
-1.61158963844
0.0738473536506
-0.163822781612
0.154148358962
0.179975812132
0.0422794978624
0.096246637762
0.209289729455
0.347671828862
0.317907489531
0.737644987991
-0.0094240208828
0.954511857257
0.396890954544
0.129830983021
0.544482942014
0.012165290159
0.0873903885034
0.139655409436
0.329867188957
0.329867188957
0.18053529147
0.140732855445
-0.0325979269933
-0.483910088726
0.237996400751
0.693684237205
-0.0187941792063
-0.740719342064
-0.0745097327035
-0.0485517474379
0.65734144165
-0.72453536505
-0.451686924732
0.319548525755
-0.031540501135
-0.136533328454
0.52651676083
0.104755381872
0.123215765255
2.86622041044
0.239926764447
-0.0700772018328
1.21663538458
-0.406868128143
-1.17827458914
-0.00216767795575
-0.312604598906
0.06723456938
-0.43306952069
0.142299070608
-0.000805597348426
-0.301570797766
0.281087783014
0.640771005189
0.375156758178
-0.109120545517
-0.125380790191
0.439467533477
-0.407765186828
0.238944694214
0.00370429258687
-0.0400235932329
-1.0927

0.0211846295967
0.375651542554
-0.312604598906
-0.057689469172
0.503750159057
-0.0244428172621
-0.0606210643074
-0.33553846654
0.110490872733
0.312545882836
0.64164349361
0.408802388137
0.2042090521
0.337652321628
0.41094413349
2.70716910079
0.372145370726
0.654431999307
-1.44012116839
-0.208060139757
0.764049614519
1.18067134333
-4.1622072332
0.10338887444
0.333684358287
0.397213597809
-0.0280968688446
0.259610780322
-0.266156412922
0.0868206769323
0.581882703401
-0.154515197927
-0.0696167411265
-0.411609445898
-0.0400235932329
0.245223626701
0.711581312603
0.024478696356
0.00348523338763
-0.00131056647928
0.169873697495
1.31323136089
1.12194406665
0.737496545601
0.302958176413
-1.97826698586
-0.186894525727
0.482257723366
0.116402337415
0.733527683095
0.134202254731
0.503659318873
0.145538160552
-0.00131056647928
0.350508744653
-0.396139775294
0.188424384816
0.0118018738307
0.204573254989
-0.0779364050051
0.127105624039
0.246601491776
0.407367948355
0.377874544056
-1.097158304
0.2092

-0.578848481498
-0.461290845363
0.10338887444
-0.543450075707
0.126134896552
-0.480235055646
0.483503366744
0.230593436682
0.177212469833
0.838197641147
0.06723456938
-0.604726081727
0.215015459128
0.333684358287
0.18902944447
-0.0606210643074
-0.000995595727529
-0.732767889102
-0.757149699211
-0.242695850232
0.456363962746
0.2372735318
0.118161635143
-0.32240797781
-0.0900606345426
-0.00774299804695
-1.4989979802
0.751046518079
0.423488375269
0.426644278674
-0.604726081727
0.0485037261456
-0.36641772525
0.00919252918516
-1.11506311137
1.46928845827
0.272078451635
-0.717095093895
0.19542626895
0.340853202544
0.528988771359
0.225958642005
-0.33553846654
0.123215765255
0.0751657898108
-1.51078102097
0.218440271215
-0.163822781612
0.477851285532
3.03453747618
-0.163836416697
0.387839793243
-0.072870988567
0.207871815241
0.352691139966
-0.251231655556
0.542136259691
0.792777192793
-1.38567520418
0.247984329115
0.347928032737
-0.596739843478
0.295433453176
-0.311049492152
0.019319415189
0.9

0.263044361113
0.346014019305
0.180475788012
0.387839793243
0.247984329115
0.199434228932
1.1029071394
0.289984375372
0.116402337415
-0.157509501244
-0.0863507832997
0.569087065447
-0.166914882837
-0.12134086632
-2.0492146807
1.17034224895
0.361636497893
0.751046518079
-0.756597036263
0.415189950014
-0.680044789121
0.0273767164485
-0.109120545517
0.257336859495
-0.100892548327
0.221807608046
0.973777669254
0.439467533477
-0.568190861315
0.169682078751
-0.26838529675
0.384237019755
0.415185179728
0.304513945628
1.06969650929
0.806674137552
0.219061724739
0.333684358287
0.319548525755
-0.0819317268527
0.4147678633
0.693684237205
0.0657986682591
0.649492426627
-0.364743311606
-0.0454511053473
-0.407765186828
-0.313562437881
-0.0444863465124
0.465863214515
0.620271556209
0.0657986682591
0.240991596773
0.319288529884
-1.70623257945
-0.676799209085
0.429450799816
0.150718506118
1.0176506683
-0.368238127155
1.18067134333
0.387187859037
0.502972685079
0.663425213017
0.187302752185
0.0669840106

2.16162536297
-0.568190861315
0.653775479243
-0.568190861315
-0.253963689034
0.913499339305
-0.154515197927
0.170147700724
0.119260259655
0.640771005189
0.154007018222
0.154767372835
0.574818464386
-0.0657613127349
-0.530786681057
-0.967013595375
0.195846884716
0.1721959724
0.0955023329584
1.23296397499
0.575237498366
0.436299561812
0.22695658555
-0.0103709442624
0.293245410043
0.111550055459
0.145538160552
0.357036542393
-0.163822781612
-0.0280968688446
0.0729796592241
-0.00118290206052
0.54269943752
0.322151943203
0.347671828862
0.436269250686
-0.208060139757
0.316539577342
0.753496699702
-0.0172633465841
-0.0134455091759
-0.160784724475
1.76694021842
0.2372735318
-2.19669870021
-0.530786681057
0.296183122134
0.363326977117
-0.290267035748
0.176631712189
0.0797008757755
-0.604726081727
0.115844777793
1.35095785411
0.387839793243
1.47038619709
-0.541480350401
0.20783991512
0.544482942014
0.61897150617
-0.221269746648
0.319548525755
-0.00691837559386
-0.00307421598497
-1.47212743059
0.

In [35]:
print(positive_pairs_dataset[0][1].shape)
print(pairs_df['FirstImage'][1][:3])
a = []
a += ['hello']
a += ['world']
print(a)

(9,)
im2
['hello', 'world']


In [48]:
pairs_list =    np.genfromtxt('Hidden Test Folder/PGMHiddenTest/PGMTestPairs.csv',delimiter=',')
pairs_df   =    pd.read_csv('Hidden Test Folder/PGMHiddenTest/PGMTestPairs.csv')
print(pairs_df)
print('--------------')
test_data_list =    np.genfromtxt('Hidden Test Folder/PGMHiddenTest/PGMTestData.csv',delimiter=',')
test_data_df   =    pd.read_csv('Hidden Test Folder/PGMHiddenTest/PGMTestData.csv')
print(test_data_df)

   Unnamed: 0 FirstImage SecondImage
0           0    im0.png     im1.png
1           1    im2.png     im7.png
2           2    im4.png     im5.png
3           3    im3.png     im2.png
4           4    im4.png     im2.png
5           5    im9.png     im3.png
--------------
   Unnamed: 0 ImageId  f1  f2  f3  f4  f5  f6  f7  f8  f9
0           0     im0   2   1   1   0   2   2   0   2   2
1           1     im1   2   1   1   0   2   2   0   1   2
2           2     im2   2   1   1   3   2   2   0   2   2
3           3     im3   1   1   1   0   2   2   0   2   2
4           4     im4   2   1   1   0   2   2   0   0   2
5           5     im5   2   1   1   3   2   2   0   1   2
6           6     im6   2   1   1   0   2   2   0   3   2
7           7     im7   1   1   1   0   2   2   0   1   2
8           8     im8   1   1   1   3   2   2   0   2   2
9           9     im9   2   1   1   3   2   2   0   0   2


In [54]:
result = classify(feat[pairs_df['FirstImage'][0][:3]],feat[pairs_df['FirstImage'][0][:3]])
print(result)

[1, 0.14532014768634233, 6.5264304292721018e-06, 5.6437011968136916e-06]


In [44]:
feat = {}
i=0
for name in test_data_df['ImageId']:
        feat[name] = np.array(test_data_df.values[i][2:])
        i+=1
print(feat)
test_pairs= []
for i in range(pairs_df.shape[0]):
        img1 = feat[pairs_df['FirstImage'][i][:3]]
        img2 = feat[pairs_df['SecondImage'][i][:3]]
        test_pairs +=[[img1,img2]]
test_pairs = np.array(test_pairs)
pred = []
S_or_D = []
LLR = []
for samples in test_pairs:
    result = classify(samples[0],samples[1])
    pred   += [result[0]]
    LLR    += [result[1]]
    if pred == 1:
        S_or_D += ['Different']
    else: S_or_D += ['Same']
pairs_df['SameOrDifferent'] = S_or_D
pairs_df['LLR'] = LLR
print(pairs_df)

{'im7': array([1, 1, 1, 0, 2, 2, 0, 1, 2], dtype=object), 'im0': array([2, 1, 1, 0, 2, 2, 0, 2, 2], dtype=object), 'im3': array([1, 1, 1, 0, 2, 2, 0, 2, 2], dtype=object), 'im4': array([2, 1, 1, 0, 2, 2, 0, 0, 2], dtype=object), 'im5': array([2, 1, 1, 3, 2, 2, 0, 1, 2], dtype=object), 'im8': array([1, 1, 1, 3, 2, 2, 0, 2, 2], dtype=object), 'im1': array([2, 1, 1, 0, 2, 2, 0, 1, 2], dtype=object), 'im2': array([2, 1, 1, 3, 2, 2, 0, 2, 2], dtype=object), 'im9': array([2, 1, 1, 3, 2, 2, 0, 0, 2], dtype=object), 'im6': array([2, 1, 1, 0, 2, 2, 0, 3, 2], dtype=object)}
   Unnamed: 0 FirstImage SecondImage SameOrDifferent       LLR
0           0    im0.png     im1.png            Same  0.173190
1           1    im2.png     im7.png            Same  0.384843
2           2    im4.png     im5.png            Same  0.497097
3           3    im3.png     im2.png            Same  0.158181
4           4    im4.png     im2.png            Same  0.469227
5           5    im9.png     im3.png            Sam